In [ ]:
from domain.entities.pet import Pet
import instructor 
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(".env")

True

In [46]:
from domain.entities.pet import Pet
from domain.attributes.pet_characteristic import PetCharacteristic
from domain.constants.attributes import (
    ActivityLevel, Size, Temperament, FurType, IntelligenceLevel,
    MaintenanceLevel, Sex
)
pet = Pet(
    name="Buddy",
    species="Dog",
    breed="Labrador Retriever",
    sex=Sex.MALE,
    age=3,
    petChar=PetCharacteristic(
        activity_level=ActivityLevel.HIGH,
        size=Size.LARGE,
        temperament=Temperament.CALM,
        fur_type=FurType.SHORT,
        intelligence=IntelligenceLevel.HIGH,
        maintenance=MaintenanceLevel.MEDIUM,
        breed="Labrador Retriever",
        species="Dog",
        hypoallergenic="N"
    )
)

In [ ]:

openai_client = OpenAI(api_key=os.getenv("OPEN_AI_KEY"))

client = instructor.from_openai(openai_client)

# Extract structured data from natural language
res = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=Pet,
    messages=[{"role": "user", "content": "Max is a corgie who plays with a ball. He loves to cuddle. Max is small. He smells good. He loves treats."}],
)


In [ ]:

with open("output.json", "w") as f:
    f.write(res.model_dump_json(indent=4))
